In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 9.6 MB/s eta 0:00:00


In [ ]:
# Necessary packages
import pathlib
import textwrap
import json

import google.generativeai as genai
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from IPython.display import display
from IPython.display import Markdown
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from google.colab import userdata

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')

In [ ]:
# Muat file Excel
df = pd.read_excel('/content/Sentimen_5.xlsx')

In [ ]:
df.head(2)

,Sentimen,True_Label
0,Wisatawan akan lebih mudah mendapat fasilitas ...,2
1,Dukungan saya terhadap kebijakan pariwisata sa...,2


In [ ]:
df['pred_label'] = ''

In [ ]:
df.head(2)

,Sentimen,True_Label,pred_label
0,Wisatawan akan lebih mudah mendapat fasilitas ...,2,
1,Dukungan saya terhadap kebijakan pariwisata sa...,2,


In [ ]:
# Convert the DataFrame to JSON using the to_json() method

json_data = df[['Sentimen','pred_label']].to_json(orient='records')

# Print the JSON data
print(json_data)

[{"Sentimen":"Wisatawan akan lebih mudah mendapat fasilitas yang nyaman","pred_label":""},{"Sentimen":"Dukungan saya terhadap kebijakan pariwisata sangat kuat.","pred_label":""},{"Sentimen":"Dukungan saya untuk memberikan pengalaman keagamaan dan memperkuat silaturahmi di antara wisatawan Muslim.","pred_label":""},{"Sentimen":"Memudahkan wisatawan muslim mencari fasilitas halal sesuai syariat Islam.","pred_label":""},{"Sentimen":"mendukung untuk dioptimalkan","pred_label":""},{"Sentimen":"Saya mendukung kebijakan pariwisata halal untuk mencegah kemaksiatan.","pred_label":""},{"Sentimen":"Saya mendukung kebijakan pariwisata halal karena memberikan rasa nyaman dan aman bagi pengunjung.","pred_label":""},{"Sentimen":" kebijakan pariwisata halal untuk meningkatkan keagamaan dan menciptakan ikon unik bagi wisatawan.","pred_label":""},{"Sentimen":"Setuju untuk menjaga salat berjamaah dan mengonsumsi makanan halal saat berlibur.","pred_label":""},{"Sentimen":"kebijakan pariwisata halal untuk 

In [ ]:
prompt = f"""
Anda adalah ahli bahasa yang ahli dalam mengklasifikasikan sentimen opini masyarakat ke dalam label Positif, Netral atau Negatif.
Bantu saya mengklasifikasikan opini masyarakat terhadap kebijakan pariwisata halal menjadi: Positif (label= 2) Netral (label = 0)dan Negatif (label = 1).
Dalam keluaran Anda, hanya kembalikan kode Json kembali sebagai keluaran - yang disediakan di antara tiga kutu balik.
Tugas Anda adalah memperbarui label yang diprediksi di bawah 'pred_label' dalam kode Json.
Jangan membuat perubahan apapun pada format kode Json.

```
{json_data}
```
"""

print(prompt)


Anda adalah ahli bahasa yang ahli dalam mengklasifikasikan sentimen opini masyarakat ke dalam label Positif, Netral atau Negatif.
Bantu saya mengklasifikasikan opini masyarakat terhadap kebijakan pariwisata halal menjadi: Positif (label= 2) Netral (label = 0)dan Negatif (label = 1).
Dalam keluaran Anda, hanya kembalikan kode Json kembali sebagai keluaran - yang disediakan di antara tiga kutu balik.
Tugas Anda adalah memperbarui label yang diprediksi di bawah 'pred_label' dalam kode Json.
Jangan membuat perubahan apapun pada format kode Json.

```
[{"Sentimen":"Wisatawan akan lebih mudah mendapat fasilitas yang nyaman","pred_label":""},{"Sentimen":"Dukungan saya terhadap kebijakan pariwisata sangat kuat.","pred_label":""},{"Sentimen":"Dukungan saya untuk memberikan pengalaman keagamaan dan memperkuat silaturahmi di antara wisatawan Muslim.","pred_label":""},{"Sentimen":"Memudahkan wisatawan muslim mencari fasilitas halal sesuai syariat Islam.","pred_label":""},{"Sentimen":"mendukung un

In [ ]:
response = model.generate_content(prompt)

print(response.text)

```
[{"Sentimen":"Wisatawan akan lebih mudah mendapat fasilitas yang nyaman","pred_label":2},{"Sentimen":"Dukungan saya terhadap kebijakan pariwisata sangat kuat.","pred_label":2},{"Sentimen":"Dukungan saya untuk memberikan pengalaman keagamaan dan memperkuat silaturahmi di antara wisatawan Muslim.","pred_label":2},{"Sentimen":"Memudahkan wisatawan muslim mencari fasilitas halal sesuai syariat Islam.","pred_label":2},{"Sentimen":"mendukung untuk dioptimalkan","pred_label":2},{"Sentimen":"Saya mendukung kebijakan pariwisata halal untuk mencegah kemaksiatan.","pred_label":2},{"Sentimen":"Saya mendukung kebijakan pariwisata halal karena memberikan rasa nyaman dan aman bagi pengunjung.","pred_label":2},{"Sentimen":" kebijakan pariwisata halal untuk meningkatkan keagamaan dan menciptakan ikon unik bagi wisatawan.","pred_label":2},{"Sentimen":"Setuju untuk menjaga salat berjamaah dan mengonsumsi makanan halal saat berlibur.","pred_label":2},{"Sentimen":"kebijakan pariwisata halal untuk menin

In [ ]:
# Clean the data by stripping the backticks
json_data = response.text.strip("`")

# Load the cleaned data and convert to DataFrame
data = json.loads(json_data)
df_sample = pd.DataFrame(data)

df_sample

,Sentimen,pred_label
0,Wisatawan akan lebih mudah mendapat fasilitas ...,2
1,Dukungan saya terhadap kebijakan pariwisata sa...,2
2,Dukungan saya untuk memberikan pengalaman keag...,2
3,Memudahkan wisatawan muslim mencari fasilitas ...,2
4,mendukung untuk dioptimalkan,2
5,Saya mendukung kebijakan pariwisata halal untu...,2
6,Saya mendukung kebijakan pariwisata halal kare...,2
7,kebijakan pariwisata halal untuk meningkatkan...,2
8,Setuju untuk menjaga salat berjamaah dan mengo...,2
9,kebijakan pariwisata halal untuk meningkatkan ...,2


In [ ]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

df['pred_label'] = df_sample['pred_label'].values
df

,Sentimen,True_Label,pred_label
0,Wisatawan akan lebih mudah mendapat fasilitas ...,2,2
1,Dukungan saya terhadap kebijakan pariwisata sa...,2,2
2,Dukungan saya untuk memberikan pengalaman keag...,2,2
3,Memudahkan wisatawan muslim mencari fasilitas ...,2,2
4,mendukung untuk dioptimalkan,2,2
5,Saya mendukung kebijakan pariwisata halal untu...,2,2
6,Saya mendukung kebijakan pariwisata halal kare...,2,2
7,kebijakan pariwisata halal untuk meningkatkan...,2,2
8,Setuju untuk menjaga salat berjamaah dan mengo...,2,2
9,kebijakan pariwisata halal untuk meningkatkan ...,2,2


In [ ]:
# prompt: Menggunakan df DataFrame: saya ingin menyimpan file kedalam bentuk excel

df.to_excel('data 5.xlsx')